## Fisrst draft 

In [51]:
%pwd

'/home/sebair/CEU-Capstone-Project'

In [197]:
#importing dependecies
import pandas as pd
import numpy as np

I am using the spredsheet from national grid which can be found 
<a href = "https://www.nationalgrideso.com/document/5073 /download"> here </a> and I took screenshot of the spreadsheet, which can be seen below. I will use pandas for reading, so I will use the pandas read_excel function, however I will need to define the sheet_names, each sheet is for one year data .

<img src = "pics/worksheet.png" alt = "pic from screenshot">


From here we can see that the 2005 t0 2009 the sheet names are without the keyword "Dem"

In [56]:
#creating names
years = ["Dem "+ str(i)+"-"+str(i%100 + 1) for i in range(2005,2020)]

In [57]:
for i in range(len(years)):
    #last two digits of the years list
    temp = years[i].split("-")
    #the sheet names are of the form 2005-06
    if int(temp[1]) < 10:
        #the sheet for the years between 2005 and 2008 they
        #do not have the string "Dem"
        new_sheet_name = temp[0].split(" ")[1] + "-0" + temp[1]
        years[i] = new_sheet_name

In [58]:
years

['2005-06',
 '2006-07',
 '2007-08',
 '2008-09',
 'Dem 2009-10',
 'Dem 2010-11',
 'Dem 2011-12',
 'Dem 2012-13',
 'Dem 2013-14',
 'Dem 2014-15',
 'Dem 2015-16',
 'Dem 2016-17',
 'Dem 2017-18',
 'Dem 2018-19',
 'Dem 2019-20']

I think these names should be fine

## Now I will only read the demand tariffs

In [307]:
sheets = pd.read_excel("TNUoS Tariffs 2005-06 to 2019-20.xlsx",
                   sheet_name= years,
                        # skip the first 5 rows because they are empty anyway
                       skiprows=5)

Each sheet is pandas dataframe, this is one example.

In [308]:
#this is for 2005
sheets[years[0]]

,Zone No.,Zone Name,Zonal Tariff (£/kW),Unnamed: 3,Zone No..1,Zone Name.,HH Zonal Tariff (£/kW),NHH Zonal Tariff (p/kWh)
0,1,Peterhead,18.162236,NaN,1.0,Northern Scotland,0.041110,0.005610
1,2,North Scotland,20.929759,NaN,2.0,Southern Scotland,4.114438,0.561693
2,3,Skye,23.095483,NaN,3.0,Northern,7.393664,0.970234
3,4,Western Highland,18.920247,NaN,4.0,North West,11.137060,1.461966
4,5,Central Highlands,15.360647,NaN,5.0,Yorkshire,11.182059,1.487585
5,6,Cruachan,15.852828,NaN,6.0,N Wales & Mersey,11.210216,1.512416
6,7,Argyll,13.441972,NaN,7.0,East Midlands,13.465848,1.804975
7,8,Stirlingshire,12.610665,NaN,8.0,Midlands,15.026957,2.062601
8,9,South Scotland,11.820471,NaN,9.0,Eastern,14.028455,1.909865
9,10,North East England,8.090616,NaN,10.0,South Wales,18.315906,2.368863


From 2005 to 2009 the demand and generator tariffs(I assume, I will need a second check from you) are on the same spreadsheet so I have to separate them(filter them out)

Below is the dataframe for 2005

In [419]:
sheets = pd.read_excel("TNUoS Tariffs 2005-06 to 2019-20.xlsx",
                   sheet_name= years,
                        # skip the first 5 rows because they are empty anyway
                       skiprows=5)
for i,j in sheets.items():
    try:
        if int(i.split("-")[0]) <= 2008:
            print("This is for",i)
            #print( sheets[i][sheets[i].columns[2:]][0:35])
#            print("---------------------------------------")
            temp = sheets[i][sheets[i].columns[2:]][0:35]
            sheets[i] = temp
            #new columns as year
            sheets[i]["year"] = i.split("-")[0]
            #new column for generator adjustment
            hh_gen_adj = sheets[i].loc[15,"HH Zonal Tariff (£/kW)"]
            nhh_gen_adj = sheets[i].loc[15,"NHH Zonal Tariff (p/kWh)"]
            sheets[i]["HH_gen_adj"] = hh_gen_adj
            sheets[i]["NHH_gen_adj"] = nhh_gen_adj
            #adding the residual
            if  int(i.split("-")[0]) == 2005:
                sheets[i]["residual"] = sheets[i].loc[sheets[i].shape[0]-1,"Zonal Tariff (£/kW)"]
            else:
                sheets[i]["residual"] = sheets[i].loc[sheets[i].shape[0]-1,"Zone No..1"]
            
            sheets[i] = sheets[i][sheets[i].columns[3:]][1:14]

    #this acts as the embeded else, because it gets me the error
    #for invalid itteral(meaning that I have strings and I see
    #it as number)
    except Exception as e:
        #print("excepption")
        #year = i.split(" ")[0]
        print(e)
        pass
        

This is for 2005-06
This is for 2006-07
This is for 2007-08
This is for 2008-09
invalid literal for int() with base 10: 'Dem 2009'
invalid literal for int() with base 10: 'Dem 2010'
invalid literal for int() with base 10: 'Dem 2011'
invalid literal for int() with base 10: 'Dem 2012'
invalid literal for int() with base 10: 'Dem 2013'
invalid literal for int() with base 10: 'Dem 2014'
invalid literal for int() with base 10: 'Dem 2015'
invalid literal for int() with base 10: 'Dem 2016'
invalid literal for int() with base 10: 'Dem 2017'
invalid literal for int() with base 10: 'Dem 2018'
invalid literal for int() with base 10: 'Dem 2019'


Now the dataframes look like this

In [320]:
sheets["2005-06"]

,Zone Name.,HH Zonal Tariff (£/kW),NHH Zonal Tariff (p/kWh),year,HH_gen_adj,NHH_gen_adj,residual
1,Southern Scotland,4.114438,0.561693,2005,0.04111,0.00561,11.19
2,Northern,7.393664,0.970234,2005,0.04111,0.00561,11.19
3,North West,11.137060,1.461966,2005,0.04111,0.00561,11.19
4,Yorkshire,11.182059,1.487585,2005,0.04111,0.00561,11.19
5,N Wales & Mersey,11.210216,1.512416,2005,0.04111,0.00561,11.19
6,East Midlands,13.465848,1.804975,2005,0.04111,0.00561,11.19
7,Midlands,15.026957,2.062601,2005,0.04111,0.00561,11.19
8,Eastern,14.028455,1.909865,2005,0.04111,0.00561,11.19
9,South Wales,18.315906,2.368863,2005,0.04111,0.00561,11.19
10,South East,15.989410,2.167559,2005,0.04111,0.00561,11.19


In [321]:
sheets["2006-07"]

,Zone Name.,HH Zonal Tariff (£/kW),NHH Zonal Tariff (p/kWh),year,HH_gen_adj,NHH_gen_adj,residual
1,Southern Scotland,5.489199,0.727452,2006,0.051802,0.006991,12.42538
2,Northern,8.492636,1.149576,2006,0.051802,0.006991,12.42538
3,North West,12.026138,1.605120,2006,0.051802,0.006991,12.42538
4,Yorkshire,12.046757,1.646536,2006,0.051802,0.006991,12.42538
5,N Wales & Mersey,12.270131,1.645144,2006,0.051802,0.006991,12.42538
6,East Midlands,14.562806,1.993682,2006,0.051802,0.006991,12.42538
7,Midlands,16.103361,2.196657,2006,0.051802,0.006991,12.42538
8,Eastern,15.141222,2.107075,2006,0.051802,0.006991,12.42538
9,South Wales,19.768764,2.573303,2006,0.051802,0.006991,12.42538
10,South East,17.948403,2.481972,2006,0.051802,0.006991,12.42538


In [322]:
sheets["2007-08"]

,Zone Name.,HH Zonal Tariff (£/kW),NHH Zonal Tariff (p/kWh),year,HH_gen_adj,NHH_gen_adj,residual
1,Southern Scotland,6.362303,0.830136,2007,0.055127,0.00709,14.061505
2,Northern,9.884146,1.287148,2007,0.055127,0.00709,14.061505
3,North West,13.646168,1.734890,2007,0.055127,0.00709,14.061505
4,Yorkshire,13.615270,1.750626,2007,0.055127,0.00709,14.061505
5,N Wales & Mersey,14.084355,1.805802,2007,0.055127,0.00709,14.061505
6,East Midlands,16.370802,2.129626,2007,0.055127,0.00709,14.061505
7,Midlands,17.807318,2.301762,2007,0.055127,0.00709,14.061505
8,Eastern,17.060375,2.240442,2007,0.055127,0.00709,14.061505
9,South Wales,21.537451,2.713949,2007,0.055127,0.00709,14.061505
10,South East,20.076054,2.586190,2007,0.055127,0.00709,14.061505


In [323]:
sheets["2008-09"]

,Zone Name.,HH Zonal Tariff (£/kW),NHH Zonal Tariff (p/kWh),year,HH_gen_adj,NHH_gen_adj,residual
1,Southern Scotland,7.950377,1.049619,2008,0.061298,0.008062,15.403402
2,Northern,10.899023,1.424364,2008,0.061298,0.008062,15.403402
3,North West,14.911638,1.956780,2008,0.061298,0.008062,15.403402
4,Yorkshire,14.829178,1.935788,2008,0.061298,0.008062,15.403402
5,N Wales & Mersey,15.460156,2.036959,2008,0.061298,0.008062,15.403402
6,East Midlands,17.620712,2.343048,2008,0.061298,0.008062,15.403402
7,Midlands,19.147817,2.543378,2008,0.061298,0.008062,15.403402
8,Eastern,18.364952,2.474431,2008,0.061298,0.008062,15.403402
9,South Wales,23.230707,2.940514,2008,0.061298,0.008062,15.403402
10,South East,21.500870,2.849708,2008,0.061298,0.008062,15.403402


I have to fix the sheets that starts with "Dem"(all the sheets later than 2009)

In [365]:
kk = sheets["Dem 2009-10"][sheets["Dem 2009-10"].columns[0:5]]
kk

,Zone No.,Zone Name.,HH Zonal Tariff (£/kW),NHH Zonal Tariff (p/kWh),Unnamed: 4
0,1,Northern Scotland,3.37924,0.459137,NaN
1,2,Southern Scotland,9.06643,1.138085,NaN
2,3,Northern,12.0594,1.598288,NaN
3,4,North West,16.5429,2.095188,NaN
4,5,Yorkshire,16.2979,2.126680,NaN
5,6,N Wales & Mersey,16.8887,2.028429,NaN
6,7,East Midlands,19.1302,2.469531,NaN
7,8,Midlands,20.5266,2.705483,NaN
8,9,Eastern,20.0057,2.526505,NaN
9,10,South Wales,23.6837,2.938924,NaN


In [ ]:
kk.loc(15)

In [364]:
kk = sheets["Dem 2011-12"][sheets["Dem 2011-12"].columns[0:5]]
kk

,Zone No.,Zone Name.,HH Zonal Tariff (£/kW),NHH Zonal Tariff (p/kWh),Unnamed: 4
0,1,Northern Scotland,6.5354,0.886871,NaN
1,2,Southern Scotland,11.7306,1.666105,NaN
2,3,Northern,15.6848,2.170176,NaN
3,4,North West,19.4492,2.736856,NaN
4,5,Yorkshire,19.583,2.704505,NaN
5,6,N Wales & Mersey,20.2046,2.952899,NaN
6,7,East Midlands,22.2054,3.095769,NaN
7,8,Midlands,23.8114,3.387267,NaN
8,9,Eastern,22.6717,3.127405,NaN
9,10,South Wales,22.8462,3.097454,NaN


In [420]:
import re
for i,j in sheets.items():
    if i.startswith("Dem"):
        year= i.split(" ")[1].split("-")[0]
        print("year",year)
        print("For sheet",i,"the shape is",sheets[i].shape)
        #Until 2012 its fine
        if(int(year) <= 2012 ):
            print("The HH Zonal tariff for small gen",sheets[i].iloc[15,2])
            hh_zonal = sheets[i].iloc[15,2]
            print("The NHH Zonal tariff for small gen",sheets[i].iloc[15,3])
            nhh_zonal = sheets[i].iloc[15,3]
            print("The residual demand tariff is  ", sheets[i].iloc[18,4])
            res_demand = sheets[i].iloc[18,4]
            sheets[i]["hh_small_gen_tariff"] = hh_zonal
            sheets[i]["nhh_small_gen_tariff"] = nhh_zonal
            sheets[i]["residual_demand"] = res_demand
            
            print("__________________________________")
        elif int(year) >= 2013 and int(year) <= 2016:
            print("The HH Zonal tariff for small gen",sheets[i].iloc[15,3])
            sheets[i]["hh_small_gen_tariff"] = sheets[i].iloc[15,3]
            print("The NHH Zonal tariff for small gen",sheets[i].iloc[15,4])
            sheets[i]["nhh_small_gen_tariff"] =  sheets[i].iloc[15,4]
            print("The residual demand tariff is  ", sheets[i].iloc[18,4])
            sheets[i]["residual_demand"] = sheets[i].iloc[18,4]
            print("__________________________________")
        else:
            #it contains characters, but I excluded the pound sign
            print("The HH Zonal tariff for small gen",re.sub('[^0-9,.]', '', sheets[i].iloc[15,3]))
            sheets[i]["hh_small_gen_tariff"] = re.sub('[^0-9,.]', '', sheets[i].iloc[15,3])
            print("The NHH Zonal tariff for small gen",re.sub('[^0-9,.]', '', sheets[i].iloc[15,4]))
            sheets[i]["nhh_small_gen_tariff"] = re.sub('[^0-9,.]', '', sheets[i].iloc[15,4])
            print("The residual demand tariff is  ", sheets[i].iloc[18,4])
            sheets[i]["residual_demand"] = re.sub('[^0-9,.]', '', str(sheets[i].iloc[18,4]))
            print("__________________________________")
            

year 2009
For sheet Dem 2009-10 the shape is (20, 5)
The HH Zonal tariff for small gen 0.0893
The NHH Zonal tariff for small gen 0.011304
The residual demand tariff is   16.943693697313442
__________________________________
year 2010
For sheet Dem 2010-11 the shape is (21, 5)
The HH Zonal tariff for small gen 0.12380338289354835
The NHH Zonal tariff for small gen 0.01704973923585892
The residual demand tariff is   18.558559862804156
__________________________________
year 2011
For sheet Dem 2011-12 the shape is (20, 5)
The HH Zonal tariff for small gen 0.12905793587548497
The NHH Zonal tariff for small gen 0.01800602129396801
The residual demand tariff is   3.60793266095396
__________________________________
year 2012
For sheet Dem 2012-13 the shape is (20, 5)
The HH Zonal tariff for small gen 0.1666147351910715
The NHH Zonal tariff for small gen 0.023366148085556863
The residual demand tariff is   22.82837287031424
__________________________________
year 2013
For sheet Dem 2013-14 the

In [422]:
sheets["Dem 2019-20"]

,Unnamed: 0,Zone,Zone Name,HH Demand Tariff\n(£/kW),NHH Demand Tariff\n(p/kWh),Embedded Export Tariff (£/kW),hh_small_gen_tariff,nhh_small_gen_tariff,residual_demand
0,NaN,1,Northern Scotland,20.9713,2.82045,0.000000,0.618659,0.080795,50.78039
1,NaN,2,Southern Scotland,30.7554,4.02604,0.000000,0.618659,0.080795,50.78039
2,NaN,3,Northern,41.0267,5.21383,7.604902,0.618659,0.080795,50.78039
3,NaN,4,North West,47.8316,6.20228,14.409800,0.618659,0.080795,50.78039
4,NaN,5,Yorkshire,48.0393,6.11633,14.617537,0.618659,0.080795,50.78039
5,NaN,6,N Wales & Mersey,49.3454,6.22376,15.923587,0.618659,0.080795,50.78039
6,NaN,7,East Midlands,51.4398,6.73856,18.017989,0.618659,0.080795,50.78039
7,NaN,8,Midlands,52.9281,6.97743,19.506286,0.618659,0.080795,50.78039
8,NaN,9,Eastern,53.7883,7.49669,20.366546,0.618659,0.080795,50.78039
9,NaN,10,South Wales,49.7256,5.87329,16.303862,0.618659,0.080795,50.78039


Because I have only 14 zones I will only get the first 14 rows of every dataframe

In [428]:
for i,j in sheets.items():
    sheets[i] = sheets[i][:14]

In [433]:
sheets["Dem 2012-13"]

,Unnamed: 0,Zone No.,Zone Name,HH Zonal Tariff (£/kW),NHH Zonal Tariff (p/kWh),hh_small_gen_tariff,nhh_small_gen_tariff,residual_demand
0,NaN,1.0,Northern Scotland,23.469195,3.388532,0.35089,0.047929,35.634955
1,NaN,2.0,Southern Scotland,26.789320,3.559740,0.35089,0.047929,35.634955
2,NaN,3.0,Northern,32.617844,4.283661,0.35089,0.047929,35.634955
3,NaN,4.0,North West,35.683316,4.874799,0.35089,0.047929,35.634955
4,NaN,5.0,Yorkshire,36.287690,5.185476,0.35089,0.047929,35.634955
5,NaN,6.0,N Wales & Mersey,35.620770,5.679363,0.35089,0.047929,35.634955
6,NaN,7.0,East Midlands,39.066214,5.234958,0.35089,0.047929,35.634955
7,NaN,8.0,Midlands,39.629994,5.487374,0.35089,0.047929,35.634955
8,NaN,9.0,Eastern,41.176427,5.539798,0.35089,0.047929,35.634955
9,NaN,10.0,South Wales,37.608777,5.245539,0.35089,0.047929,35.634955


# Quesion 1: In the spreadsheet there are demand and generator charges for years 2005 to 2008. For now the HH zonal tariff and NHH zonal tariff will be inputs. Which one should I take for the residual?

In [217]:
math.isnan()

TypeError: isnan() takes exactly one argument (0 given)